# Some Title

In [302]:
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
import requests
import json
import os

In [150]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

In [151]:
openai = OpenAI()

In [292]:
class Website:

    def __init__(self, url):
        self.url = url

        data = requests.get(self.url).text

        soup = BeautifulSoup(data, 'html.parser')

        self.title = soup.title.string if soup.title else 'No title found.'

        if soup.body:
            for irrelevant in soup.body(['script', 'style', 'img', 'input']):
                irrelevant.decompose()

            text = soup.body.get_text(separator='\n', strip=True)

            self.text = '\n'.join([i for i in text.split('\n') if len(i.split()) >= 3])
        else:
            self.text = ''

        links = [link.get('href') for link in soup.find_all('a')]

        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"

In [308]:
url = 'https://www.anthropic.com/'

In [293]:
website = Website(url)

In [294]:
link_system_prompt = 'You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages. You should respond in JSON as in this example:'

link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [295]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages. You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [296]:
link_user_prompt = f"Here is the list of links on the website of {website.url}. \
Please decide which of these are relevant web links for a brochure about the company. \
Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, or email links. \
Note that some of these might be relative links:\n\n"

link_user_prompt += '\n'.join(website.links)

In [297]:
print(link_user_prompt)

Here is the list of links on the website of https://www.anthropic.com/. Please decide which of these are relevant web links for a brochure about the company. Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, or email links. Note that some of these might be relative links:

/
/claude
/team
/enterprise
/api
/pricing
/research
/company
/careers
/news
https://www.anthropic.com/research#entry:8@1:url
https://www.anthropic.com/claude
https://claude.ai/
/api
/news/3-5-models-and-computer-use
/claude/sonnet
/claude/haiku
/news/claude-for-enterprise
/research/constitutional-ai-harmlessness-from-ai-feedback
/news/core-views-on-ai-safety
/jobs
/
/claude
/api
/team
/pricing
/research
/company
/customers
/news
/careers
mailto:press@anthropic.com
https://support.anthropic.com/
https://status.anthropic.com/
/supported-countries
https://twitter.com/AnthropicAI
https://www.linkedin.com/company/anthropicresearch
https://www.youtube.com/@anthropic-ai
/legal/consume

In [152]:
response = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': link_system_prompt},
        {'role': 'user', 'content': link_user_prompt}
    ],
    response_format={'type': 'json_object'}
)

result = response.choices[0].message.content

In [154]:
print(result)

{
    "links": [
        {"type": "about page", "url": "https://www.anthropic.com/company"},
        {"type": "careers page", "url": "https://www.anthropic.com/careers"},
        {"type": "team page", "url": "https://www.anthropic.com/team"},
        {"type": "research page", "url": "https://www.anthropic.com/research"},
        {"type": "enterprise page", "url": "https://www.anthropic.com/enterprise"},
        {"type": "api page", "url": "https://www.anthropic.com/api"},
        {"type": "pricing page", "url": "https://www.anthropic.com/pricing"},
        {"type": "news page", "url": "https://www.anthropic.com/news"}
    ]
}


In [298]:
brochure_system_prompt = 'You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors, and recruits. Respond in markdown. \
Include details about company culture, customers, and careers/jobs if you have the information.'

In [299]:
company_name = 'Anthropic'

brochure_user_prompt = f"You are looking at website information for a company called {company_name}. \
Here are the contents of the company landing page and other relevant pages. \
Use this information to build a short brochure for the company in markdown."

In [300]:
result_json = json.loads(result)

brochure_user_prompt += f"\n\nLANDING PAGE:\n{Website(url).get_contents()}"

for link in result_json['links']:
    brochure_user_prompt += f"\n{link['type'].upper()}:\n{Website(link['url']).get_contents()}"

brochure_user_prompt = brochure_user_prompt[:20000]

In [301]:
print(brochure_user_prompt)

You are looking at website information for a company called Anthropic. Here are the contents of the company landing page and other relevant pages. Use this information to build a short brochure for the company in markdown.

LANDING PAGE:
Webpage Title:
Home \ Anthropic

Webpage Contents:
that put safety at the frontier
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Claude for Enterprise
Sep 4, 2024
Constitutional AI: Harmlessness from AI Feedback
Dec 15, 2022
Core Views on AI Safety: When, Why, What, and How
Mar 8, 2023
Work with Anthropic
Anthropic is an AI safety and research company based in San Francisco. Our interdisciplinary team has experience across ML, physics, policy, and product. Together, we generate research and create reliable, beneficial AI s

In [306]:
response = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': brochure_system_prompt},
        {'role': 'user', 'content': brochure_user_prompt}
    ]
)

result = response.choices[0].message.content

In [307]:
display(Markdown(result))

# Anthropic: Advancing AI Safety for a Better Future

## Who We Are

Anthropic is a leading AI safety and research company located in San Francisco. We are on a mission to ensure that transformative AI technology benefits humanity. Our aim is to create reliable, interpretable, and steerable AI systems, leveraging interdisciplinary expertise from fields such as machine learning, physics, policy, and product development.

### Our Products

* **Claude 3.5 Sonnet & Haiku**: Advanced AI models designed to drive efficiency and revenue streams for businesses and organizations.
* **Claude for Enterprise**: Tailored solutions that securely integrate AI into company workflows, leveraging internal knowledge for enhanced collaboration and output.

## Our Culture

At Anthropic, we prioritize a high-trust work environment that fosters collaboration and intellectual openness. Our team is diverse and passionate, comprising researchers, engineers, policy experts, and business leaders who share a commitment to AI safety. We value honesty, emotional maturity, and encourage contributions across various projects. 

### Core Principles

- **Collaboration**: We believe in teamwork across all levels, with everyone contributing to our shared mission.
- **Pragmatism**: We celebrate simple solutions and value empirical approaches that yield actionable insights.
- **Continuous Learning**: We seek to share our findings with the broader community and learn from external collaborators.

## Our Customers

We cater to a wide range of clients, including businesses, nonprofits, and civil society groups seeking to adopt safe AI solutions. With Claude, teams can enhance productivity, generate insights rapidly, and collaborate seamlessly. 

## Careers at Anthropic

We are always looking for talented individuals eager to contribute to the field of AI. We offer a dynamic work environment and a comprehensive benefits package that includes:

- **Health & Wellness**: Competitive health, dental, and vision insurance, mental health support, and inclusive fertility benefits.
- **Compensation & Support**: Competitive salary, equity packages, generous parental leave, home office stipends, and continuous growth through education stipends.
- **A Focus on Diversity**: We welcome varied experiences and backgrounds, aiming to create a truly inclusive work environment.

### Join Us!

Explore open roles on our [Careers Page](#) and be a part of a mission-driven organization making a tangible impact in the AI landscape.

---

For more detailed insights about our products, team, or research initiatives, feel free to visit our website or contact us directly. Join Anthropic as we strive for a future where AI technologies are safely integrated into society and enhance human capabilities.